# Importa bibliotecas

In [0]:
# instala antes porque não vem no spark
!pip install statsforecast;
!pip install catboost;
!pip install lightgbm;
!pip install xgboost;
!pip install pandas_ta;

In [0]:
%pip install --upgrade threadpoolctl


In [0]:
# Manipulação de dados
import pandas as pd
import numpy as np
import pandas_ta as ta
import calendar
from datetime import date

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Séries temporais
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler 
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Modelagem de séries temporais (forecast)
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA, ARIMA, Naive, SeasonalNaive,
    SeasonalWindowAverage, WindowAverage
)
from statsforecast.utils import ConformalIntervals

# Machine learning - Regressão e Classificação
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier 
from lightgbm import LGBMClassifier 
from xgboost import XGBClassifier 
from sklearn.svm import SVC 

# Métricas
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

# Importar função especifica de um módulo --> Melhores hiperparametros 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline

# Import de configuraçoes
import ipywidgets as widgets
from IPython.display import display

# Importa dataset

In [0]:
# Le a tabela
df_bovespa = spark.sql("SELECT Data As date, `Último` AS close, Abertura AS open, `Máxima` AS high, `Mínima` AS low, `Vol.` AS vol, `Var%`AS var FROM workspace.pos_fiap.dados_historicos_ibovespa ORDER BY Data")
# Converte Spark DataFrame para Pandas
df = df_bovespa.toPandas()

# Análise Exploratória

In [0]:
# Converte informações de casas decimais
def convert_to_float(value):
    if isinstance(value, str):
        value = value.replace('.', '').replace(',', '.')
        if 'B' in value:
            return float(value.replace('B', '')) * 1_000_000_000
        if 'M' in value:
            return float(value.replace('M', '')) * 1_000_000
        elif 'K' in value:
            return float(value.replace('K', '')) * 1_000
    return value

In [0]:
# Converte informações de percentuais

def convert_percentage_to_float(value):
    if isinstance(value, str) and '%' in value:
        value = value.replace('%', '').replace(',', '.')
        if '-' in value:
            value = value.replace('-', '')
            return float(value) * -1
        else:
            return float(value)
    return value

In [0]:
# Aplica conversões nos campos de colume e variação percentual
df['vol'] = df['vol'].apply(convert_to_float)
df['var'] = df['var'].apply(convert_percentage_to_float)
print(df.head())

In [0]:
# Cópia do dataset original
df_bovespa_index = df.copy()

In [0]:
# Converte campo date em index e cria campo de ds para aplicar nos modelos
df_bovespa_index.index = pd.to_datetime(df_bovespa_index.date, format="%Y-%m-%d")
df_bovespa_index = df_bovespa_index.rename(columns={'date': 'ds'})
df_bovespa_index.sort_index(inplace=True)
df_bovespa_index.head()

In [0]:
# verifica tamanho do dataset
df_bovespa_index.shape

In [0]:
# Verifica informaçöes do dataset
df_bovespa_index.info()

In [0]:
# Gerando a estatistica descritiva do data frame 
df_bovespa_index.describe()

In [0]:
#plota o gráfico de fechamento diário
plt.figure(figsize=(15, 6))
plt.plot(df_bovespa_index.index, df_bovespa_index['close'], label='Fechamento Diário', color= 'sienna')
plt.title('Fechamento - Diário do IBOVESPA')
plt.xlabel('Data')
plt.ylabel('Fechamento')

n_labels = 20  
labels = df_bovespa_index.index[::len(df_bovespa_index) // n_labels]

plt.xticks(labels, rotation=45)

plt.legend()
plt.show()

In [0]:
# Agrupa os dados por ano e conta o número de entradas.
df_bovespa_index['ds'] = pd.to_datetime(df_bovespa_index.index, format='%d.%m.%Y', dayfirst=True)
df_bovespa_index['anual_dias'] = df_bovespa_index['ds'].dt.year

days_per_year = df_bovespa_index.groupby('anual_dias').size()

print(sum(days_per_year)/20)

In [0]:
result = seasonal_decompose(df_bovespa_index['close'], model='multiplicative', period=248) # Assumindo um ano de 248 dias úteis


In [0]:
# Verifica como a série está se comportando
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(15, 12))

result.observed.plot(ax=ax1, color='sienna')
ax1.set_title("Componente Observada")
ax1.set_ylabel("Índice BOVESPA")
result.trend.plot(ax=ax2, color='sienna')
ax2.set_title("Tendência")
ax2.set_ylabel("Índice BOVESPA")
result.seasonal.plot(ax=ax3, color='sienna')
ax3.set_title("Sazonalidade")
ax3.set_ylabel("Fator Sazonal")
result.resid.plot(ax=ax4, color='sienna')
ax4.set_title("Resíduos")
ax4.set_ylabel("Erro")


# Eixo X compartilhado (datas)
for ax in (ax1, ax2, ax3, ax4):
    ax.set_xlabel("Data")

plt.tight_layout()

In [0]:
X = df_bovespa_index.close.values

In [0]:
result = adfuller(X)

print("Teste ADF")
print(f"Teste Estatístico: {result[0]}")
print(f"P-Value: {result[1]}")
print("Valores críticos:")

for key, value in result[4].items():
  print(f"\t{key}: {value}")

In [0]:
ma = df_bovespa_index.close.rolling(12).mean()

f, ax = plt.subplots()
df_bovespa_index.close.plot(ax=ax, legend=False)
ma.plot(ax=ax, legend=False, color='r')
plt.tight_layout()

In [0]:
df_log = np.log(df_bovespa_index['close'])
ma_log = df_log.rolling(12).mean()

f, ax = plt.subplots()
df_log.plot(ax=ax, legend=False)
ma_log.plot(ax=ax, legend=False, color='r')
plt.tight_layout()

In [0]:
df_s = (df_log - ma_log).dropna()

ma_s = df_s.rolling(12).mean()

std = df_s.rolling(12).std()

f, ax = plt.subplots()
df_s.plot(ax=ax, legend=False)
ma_s.plot(ax=ax, legend=False, color='r')
std.plot(ax=ax, legend=False, color='g')
plt.tight_layout()


In [0]:
df_s.head()

In [0]:
X_s = df_s.values
result_s = adfuller(X_s)

print("Teste ADF")
print(f"Teste Estatístico: {result_s[0]}")
print(f"P-Value: {result_s[1]}")
print("Valores críticos:")

for key, value in result_s[4].items():
  print(f"\t{key}: {value}")


In [0]:
df_diff = df_s.diff(1)
ma_diff = df_diff.rolling(12).mean()

std_diff = df_diff.rolling(12).std()


f, ax = plt.subplots()
df_diff.plot(ax=ax, legend=False)
ma_diff.plot(ax=ax, legend=False, color='r')
std_diff.plot(ax=ax, legend=False, color='g')
plt.tight_layout()

X_diff = df_diff.dropna().values
result_diff = adfuller(X_diff)

print("Teste ADF")
print(f"Teste Estatístico: {result_diff[0]}")
print(f"P-Value: {result_diff[1]}")
print("Valores críticos:")

for key, value in result_diff[4].items():
  print(f"\t{key}: {value}")

In [0]:
lag_acf = acf(df_diff.dropna(), nlags=25)
lag_pacf = pacf(df_diff.dropna(), nlags=25)

In [0]:
plt.plot(lag_acf)

plt.axhline(y= -1.96/(np.sqrt((len(df_diff) -1))), linestyle='--', color='gray',linewidth=0.7)
plt.axhline(y=0, linestyle='--', color='gray',linewidth=0.7)
plt.axhline(y= 1.96/(np.sqrt((len(df_diff) -1))), linestyle='--', color='gray',linewidth=0.7)

plt.title("ACF")
plt.show()

plt.plot(lag_pacf)

plt.axhline(y= -1.96/(np.sqrt((len(df_diff) -1))), linestyle='--', color='gray',linewidth=0.7)
plt.axhline(y=0, linestyle='--', color='gray',linewidth=0.7)
plt.axhline(y= 1.96/(np.sqrt((len(df_diff) -1))), linestyle='--', color='gray',linewidth=0.7)

plt.title("PACF")
plt.show()

In [0]:
# Verifica ACF e PACF
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# ACF
acf_plot = plot_acf(df_bovespa_index['close'], ax=ax1, lags=50)
ax1.set_title("Autocorrelação (ACF)")
ax1.set_ylabel("Correlação")
ax1.set_xlabel("Defasagem")

# PACF
pacf_plot = plot_pacf(df_bovespa_index['close'], ax=ax2, lags=50)
ax2.set_title("Autocorrelação Parcial (PACF)")
ax2.set_ylabel("Correlação")
ax2.set_xlabel("Defasagem")

plt.tight_layout()

In [0]:

df_bovespa_index['Data'] = pd.to_datetime(df_bovespa_index.index, format='%d.%m.%Y', dayfirst=True)
df_bovespa_index['Dia'] = df_bovespa_index['Data'].dt.day
df_bovespa_index['Mês'] = df_bovespa_index['Data'].dt.month
df_bovespa_index['Ano'] = df_bovespa_index['Data'].dt.year
df_bovespa_index['Trimestre'] = df_bovespa_index['Data'].dt.quarter
df_bovespa_index['Dia_da_Semana'] = df_bovespa_index['Data'].dt.dayofweek
df_bovespa_index['Fim_de_Semana'] = df_bovespa_index['Dia_da_Semana'].apply(lambda x: 1 if x >= 5 else 0)

In [0]:
# Criação de lags: Lags são valores defasados de uma série temporal, ou seja, os valores anteriores a um ponto no tempo. Eles são usados para capturar dependências temporais nos dados. 
# Na modelagem de séries temporais, os lags ajudam a prever o valor atual com base nos valores passados. 

df_bovespa_index['Lag_1'] = df_bovespa_index['close'].shift(1)  # shift() desloca a série temporal em n períodos, criando novas colunas.
df_bovespa_index['Lag_5'] = df_bovespa_index['close'].shift(5)
df_bovespa_index['Lag_10'] = df_bovespa_index['close'].shift(10)

# Criação de médias móveis

df_bovespa_index['MA_5'] = df_bovespa_index['close'].rolling(window=5).mean()
df_bovespa_index['MA_10'] = df_bovespa_index['close'].rolling(window=10).mean()
df_bovespa_index['MA_20'] = df_bovespa_index['close'].rolling(window=20).mean()

In [0]:
# Média Móvel Exponencial
df_bovespa_index['EMA_10'] = df_bovespa_index['close'].ewm(span=10, adjust=False).mean()
df_bovespa_index['EMA_20'] = df_bovespa_index['close'].ewm(span=20, adjust=False).mean()

In [0]:
def calcular_RSI(data, window):
    delta = data.diff(1) # diff(1): Calcula a diferença entre o preço atual e o preço anterior
    gain = delta.where(delta > 0, 0) # Mantém as diferenças positivas e define as negativas como 0.
    loss = -delta.where(delta < 0, 0) # Mantém as diferenças negativas (como valores positivos) e define as positivas como 0.
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean() # Calcula a média móvel dos ganhos e perdas.
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss # Relativa Força: ratio de ganho médio para perda média
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [0]:
df_bovespa_index['RSI_14'] = calcular_RSI(df_bovespa_index['close'], 14)

In [0]:
df_bovespa_index.head()

In [0]:
plt.figure(figsize=(14, 7))

# Preço de fechamento
plt.subplot(2, 1, 1)
plt.plot(df_bovespa_index['close'], label='Fechamento')
plt.title('Preço de Fechamento')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.legend()

# RSI
plt.subplot(2, 1, 2)
plt.plot(df_bovespa_index['RSI_14'], label='RSI 14', color='orange')
plt.axhline(y=70, color='red', linestyle='--', label='Sobrecompra (70)')
plt.axhline(y=30, color='green', linestyle='--', label='Sobrevenda (30)')
plt.title('Índice de Força Relativa (RSI)')
plt.xlabel('Data')
plt.ylabel('RSI')
plt.legend()

plt.tight_layout()
plt.show()

In [0]:

def calcular_bollinger_bands(data, window):
    MA = data.rolling(window=window).mean() # Calcula a média móvel simples.
    std = data.rolling(window=window).std() # Calcula o desvio padrão móvel
    upper_band = MA + (std * 2)
    lower_band = MA - (std * 2)
    return upper_band, lower_band


In [0]:
df_bovespa_index['Upper_BB'], df_bovespa_index['Lower_BB'] = calcular_bollinger_bands(df_bovespa_index['close'], 20)

In [0]:
plt.figure(figsize=(14, 7))
plt.plot(df_bovespa_index['close'], label='Fechamento')
plt.plot(df_bovespa_index['Upper_BB'], label='Banda Superior', linestyle='--')
plt.plot(df_bovespa_index['Lower_BB'], label='Banda Inferior', linestyle='--')
plt.fill_between(df_bovespa_index.index, df_bovespa_index['Upper_BB'], df_bovespa_index['Lower_BB'], color='gray', alpha=0.2)
plt.title('Fechamento Diário e Bandas de Bollinger')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.legend()
plt.show()

# As Bandas de Bollinger fornecem uma faixa de preço dentro da qual o ativo geralmente oscila, ajudando a identificar pontos de sobrecompra e sobrevenda. 
# Quando o preço se aproxima ou ultrapassa as bandas, pode sinalizar uma possível reversão ou continuação da tendência.

In [0]:
df_bovespa_index['var'] = df_bovespa_index['close'].pct_change() * 100 # Calcula a mudança percentual entre o preço atual e o preço anterior.

# A variação percentual diária mostra a mudança percentual no preço de fechamento de um dia para o próximo. É útil para medir a amplitude das mudanças diárias.

In [0]:
df_bovespa_index['Volatilidade_10'] = df_bovespa_index['var'].rolling(window=10).std() # Calcula o desvio padrão móvel para os últimos 10 dias, representando a volatilidade.
df_bovespa_index['Volatilidade_20'] = df_bovespa_index['var'].rolling(window=20).std()

# A volatilidade mede a variação do preço de um ativo ao longo do tempo. Ela indica o quão estável ou instável é o preço do ativo. Altos valores de volatilidade significam grandes 
# flutuações no preço, enquanto baixos valores indicam flutuações menores.


In [0]:
plt.figure(figsize=(14, 7))
plt.plot(df_bovespa_index['var'], label='Variação Percentual Diária')
plt.title('Variação Percentual Diária')
plt.xlabel('Data')
plt.ylabel('Variação (%)')
plt.legend()
plt.show()


In [0]:
plt.figure(figsize=(14, 7))

# Volatilidade de 10 dias
plt.subplot(2, 1, 1)
plt.plot(df_bovespa_index['Volatilidade_10'], label='Volatilidade 10 dias', color='blue')
plt.title('Volatilidade de 10 dias')
plt.xlabel('Data')
plt.ylabel('Volatilidade')
plt.legend()

# Volatilidade de 20 dias
plt.subplot(2, 1, 2)
plt.plot(df_bovespa_index['Volatilidade_20'], label='Volatilidade 20 dias', color='red')
plt.title('Volatilidade de 20 dias')
plt.xlabel('Data')
plt.ylabel('Volatilidade')
plt.legend()

plt.tight_layout()
plt.show()

In [0]:
df_bovespa_index = df_bovespa_index.dropna()

In [0]:
df_bovespa_index.head()

In [0]:
# Criar a variavel Target (Alvo) --> 1 para sobe e 0 para desce 
# Avaliamos o valor atual com o valor do proximo dia e dessa forma conseguimos definir se o dia seguinte subiu (1) ou desceu (0)

df_bovespa_index['target'] = (df_bovespa_index["close"].shift(-1) > df_bovespa_index["close"]).astype(int)
df_bovespa_index.tail()

In [0]:
# Copia base de dados para treinar modelos
df_bovespa_index_1 = df_bovespa_index.copy()

# Treinamento dos Modelos:


# Modelos de séries temporais

In [0]:
# DF para modelos de séries temporais
df_bovespa_series_temp = df_bovespa_index_1[['ds', 'close']].copy()

In [0]:
df_bovespa_series_temp.head()

In [0]:

df_bovespa_series_temp['unique_id'] = 'serie_unica'
df_bovespa_series_temp=df_bovespa_series_temp.rename(columns={'close':'y'})
df_bovespa_series_temp.head()

In [0]:
df_bovespa_series_temp = df_bovespa_series_temp.dropna()

In [0]:
treino = df_bovespa_series_temp.loc[df_bovespa_series_temp['ds'] < '2025-06-04']
valid = df_bovespa_series_temp.loc[(df_bovespa_series_temp['ds'] >= '2025-06-04') & (df_bovespa_series_temp['ds'] < '2025-07-17')]
h = valid['ds'].nunique()

In [0]:
h

In [0]:
def wmape(y_true, y_pred):
    return np.abs(y_true-y_pred).sum() / np.abs(y_true).sum()

## Naive

In [0]:
treino.tail()

In [0]:

model = StatsForecast(models=[Naive()], freq='D', n_jobs=-1)
model.fit(treino)

forecast_df = model.predict(h=h, level=[90])
forecast_df = forecast_df.reset_index(drop=True).merge(valid, on=['ds', 'unique_id'], how='left')
forecast_df = forecast_df.dropna(subset=['y'])  # remove onde y é NaN


wmape1 = wmape(forecast_df['y'].values, forecast_df['Naive'].values)
print(f"WMAPE: {wmape1:.2%}")
print(f'Percentual de acerto {1- wmape1:.2%}')

model.plot(treino, forecast_df, level=[90], engine ='matplotlib', max_insample_length=90)


## Arima


In [0]:
model_a = StatsForecast(models=[AutoARIMA(season_length=7)], freq='D', n_jobs=-1)
model_a.fit(treino)

forecast_arima = model_a.predict(h=h, level=[90])
forecast_arima = forecast_arima.reset_index(drop=True).merge(valid, on=['ds', 'unique_id'], how='inner')

wmape_arima = wmape(forecast_arima['y'].values, forecast_arima['AutoARIMA'].values)
print(f'wmape: {wmape_arima:.2%}')
print(f'Percentual de acerto {1- wmape_arima:.2%}')

model_a.plot(treino, forecast_arima, level=[90], engine ='matplotlib', max_insample_length=90)

## SeasonalNaive

In [0]:
model_s = StatsForecast(models=[SeasonalNaive(season_length=7)], freq='D', n_jobs=-1)
model_s.fit(treino)

forecast_dfs = model_s.predict(h=h, level=[90])
forecast_dfs = forecast_dfs.reset_index(drop=True).merge(valid, on=['ds', 'unique_id'], how='inner')

wmape2 = wmape(forecast_dfs['y'].values, forecast_dfs['SeasonalNaive'].values)
print(f"WMAPE: {wmape2:.2%}")
print(f'Percentual de acerto {1- wmape2:.2%}')

model_s.plot(treino, forecast_dfs, level=[90],engine ='matplotlib', max_insample_length=90)

## SeasonalWindowAverage

In [0]:
intervals = ConformalIntervals(h=h, n_windows=2)

model_sm = StatsForecast(models=[SeasonalWindowAverage(season_length=7, window_size=2, prediction_intervals=intervals)], freq='D', n_jobs=-1)
model_sm.fit(treino)

forecast_dfsm = model_sm.predict(h=h, level=[90])
forecast_dfsm = forecast_dfsm.reset_index(drop=True).merge(valid, on=['ds', 'unique_id'], how='inner')

wmape3 = wmape(forecast_dfsm['y'].values, forecast_dfsm['SeasWA'].values)
print(f"WMAPE: {wmape3:.2%}")
print(f'Percentual de acerto {1- wmape3:.2%}')

model_sm.plot(treino, forecast_dfsm, level=[90], engine='matplotlib', max_insample_length=90)

# Demais modelos

In [0]:
# DF para demais modelos
df_bovespa_model = df_bovespa_index.copy()

In [0]:
df_bovespa_model.head()

In [0]:
# Separa os dados para treino e teste 

lista = ["ds","open", "high", "low","target", "vol", "var", "anual_dias", "Data", "Dia", "Mês", "Ano", "Trimestre", "Dia_da_Semana", "Fim_de_Semana"]

x = df_bovespa_model.drop(columns=lista)
y = df_bovespa_model["target"].copy()

print("Features (x):")
display(x.tail())

print("\nAlvo (y):")
display(y.tail())

In [0]:
# Separa a base em treino e teste

x_train = x[x.index < '2015-06-04']
y_train = y[y.index < '2015-06-04']

x_test = x[(x.index >= '2025-06-04') & (x.index < '2025-07-17')]
y_test = y[(y.index >= '2025-06-04') & (y.index < '2025-07-17')]

In [0]:
# Prepara o TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

In [0]:
# Normaliza os dados

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [0]:
# Regressao Logistica

usar_grid = "Não"

if usar_grid == "Sim":

    pipe_lr = Pipeline(
        [('scaler', StandardScaler()), ('clf', LogisticRegression(solver='liblinear'))]
    )

    param_grid_lr = {
        'clf__C': np.logspace(-4, 3, 8),  # regularização
        'clf__penalty': ['l1', 'l2'],  # tipo de penalização
        'clf__solver': ['liblinear'],  # necessário para suportar l1
    }

    modelo_rl = GridSearchCV(pipe_lr, param_grid_lr, cv=tscv)
    modelo_rl.fit(x_train, y_train)
    modelo_rl = modelo_rl.predict(x_test)

else: 
    modelo_rl = LogisticRegression(random_state=42)
    modelo_rl.fit(x_train_scaled,y_train.ravel())
    previsao_rl = modelo_rl.predict(x_test_scaled)

In [0]:
# Random Forest 

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(x_train_scaled, y_train)
previsao_rf = modelo_rf.predict(x_test_scaled)

In [0]:
# XGBoost

usar_grid = "Não"

if usar_grid == "Sim":

    pipe_xgb = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('clf', XGBClassifier()),
        ]
    )

    param_grid_xgb = {
        'clf__n_estimators': [50, 100, 200],  # número de árvores
        'clf__max_depth': [3, 5, 7],  # profundidade da árvore
        'clf__learning_rate': [0.01, 0.05, 0.1, 0.3],  # taxa de aprendizado
        'clf__subsample': [0.6, 0.8, 1.0],  # fração das amostras usadas em cada árvore
        'clf__colsample_bytree': [0.6, 0.8, 1.0],  # fração de colunas usadas por árvore
        'clf__gamma': [0, 0.1, 0.5, 1.0],  # regularização mínima para divisão
        'clf__reg_alpha': [0, 0.1, 1],  # L1 regularization
        'clf__reg_lambda': [0.1, 1, 10],  # L2 regularization
    }

    modelo_xgb = RandomizedSearchCV(pipe_xgb, param_grid_xgb, cv=tscv)
    modelo_xgb.fit(x_train, y_train)
    previsao_xgb = modelo_xgb.predict(x_test)

else:
    modelo_xgb = XGBClassifier(random_state=42)
    modelo_xgb.fit(x_train_scaled, y_train)
    previsao_xgb = modelo_xgb.predict(x_test_scaled)

In [0]:
# LightGBM

modelo_lgbm = LGBMClassifier(random_state=42, verbose = -1)
modelo_lgbm.fit(x_train_scaled, y_train)

#previsao_lgbm = modelo_lgbm.predict(x_test_scaled)
previsao_lgbm = modelo_lgbm.predict(pd.DataFrame(x_test_scaled, columns=x_train.columns)) # --> Gerado o predict dessa forma para não gerar o Warning

In [0]:
# CatBoost

modelo_cat = CatBoostClassifier(random_state=42, verbose=0)
modelo_cat.fit(x_train_scaled, y_train)
previsao_cat = modelo_cat.predict(x_test_scaled)


In [0]:
# SVM 

modelo_svm = SVC(kernel='rbf', C=1.0, probability=True, random_state=42)
modelo_svm.fit(x_train_scaled, y_train)
previsao_svm = modelo_svm.predict(x_test_scaled)

In [0]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

def converte_direcao_binaria(y_real, y_pred):
    if len(y_real) <= 1 or len(y_pred) <= 1:
        return None, None
    
    y_real_bin = (np.diff(y_real) > 0).astype(int)
    y_pred_bin = (np.diff(y_pred) > 0).astype(int)
    
    if len(y_real_bin) != len(y_pred_bin):
        print("Erro: vetores binários de tamanhos diferentes")
        return None, None
    
    return y_real_bin, y_pred_bin

def calcula_metricas_direcao(df_forecast, col_predicao, col_real='y'):
    if col_predicao in df_forecast.columns:
        df_valid = df_forecast.dropna(subset=[col_predicao])
        y_real = df_valid[col_real].values
        y_pred = df_valid[col_predicao].values
        
        y_real_bin, y_pred_bin = converte_direcao_binaria(y_real, y_pred)
        
        if y_real_bin is not None and y_pred_bin is not None:
            acc = accuracy_score(y_real_bin, y_pred_bin)
            report = classification_report(y_real_bin, y_pred_bin, output_dict=True, zero_division=0)
            print(f"Acurácia direção {col_predicao}: {acc:.2%}")
            return report
        else:
            print(f"Poucos dados para calcular direção em {col_predicao}.")
    else:
        print(f"Coluna '{col_predicao}' não encontrada.")
    # Se não der para calcular, retorna zeros
    return {
        "accuracy": 0,
        "weighted avg": {"precision": 0, "recall": 0, "f1-score": 0}
    }

# ====================
# Calcular para cada modelo temporal
# Substitua forecast_dfarima, forecast_dfsm, forecast_dfnaive pelos seus dataframes correspondentes
# ====================

report_arima = calcula_metricas_direcao(forecast_arima, col_predicao='AutoARIMA')
report_seasnaive = calcula_metricas_direcao(forecast_dfs, col_predicao='SeasonalNaive')
report_naive = calcula_metricas_direcao(forecast_df, col_predicao='Naive')
report_seaswa = calcula_metricas_direcao(forecast_dfsm, col_predicao='SeasWA')  # já vimos esse

# ====================
# Avaliação dos classificadores tradicionais
# ====================

modelos_avaliados = {
    "Regressão Logística":                      {"modelo": modelo_rl, "previsoes": previsao_rl},
    "Random Forest":                            {"modelo": modelo_rf, "previsoes": previsao_rf},
    "XGBoost":                                  {"modelo": modelo_xgb, "previsoes": previsao_xgb},
    "LightGBM":                                 {"modelo": modelo_lgbm, "previsoes": previsao_lgbm},
    "CatBoost":                                 {"modelo": modelo_cat, "previsoes": previsao_cat},
    "SVM":                                      {"modelo": modelo_svm, "previsoes": previsao_svm},
}

performance = pd.DataFrame(columns=["Modelo", "Acurácia", "Precisão", "Recall", "F1-Score"])

for nome, dados in modelos_avaliados.items():
    report = classification_report(y_test, dados["previsoes"], output_dict=True, zero_division=0)
    performance.loc[performance.shape[0]] = {
        "Modelo": nome,
        "Acurácia": report["accuracy"],
        "Precisão": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"],
    }

# ====================
# Adicionar os modelos temporais
# ====================

performance.loc[performance.shape[0]] = {
    "Modelo": "AutoARIMA",
    "Acurácia": report_arima["accuracy"],
    "Precisão": report_arima["weighted avg"]["precision"],
    "Recall": report_arima["weighted avg"]["recall"],
    "F1-Score": report_arima["weighted avg"]["f1-score"],
}

performance.loc[performance.shape[0]] = {
    "Modelo": "SeasonalNaive",
    "Acurácia": report_seasnaive["accuracy"],
    "Precisão": report_seasnaive["weighted avg"]["precision"],
    "Recall": report_seasnaive["weighted avg"]["recall"],
    "F1-Score": report_seasnaive["weighted avg"]["f1-score"],
}

performance.loc[performance.shape[0]] = {
    "Modelo": "Naive",
    "Acurácia": report_naive["accuracy"],
    "Precisão": report_naive["weighted avg"]["precision"],
    "Recall": report_naive["weighted avg"]["recall"],
    "F1-Score": report_naive["weighted avg"]["f1-score"],
}

performance.loc[performance.shape[0]] = {
    "Modelo": "SeasonalWindowAverage",
    "Acurácia": report_seaswa["accuracy"],
    "Precisão": report_seaswa["weighted avg"]["precision"],
    "Recall": report_seaswa["weighted avg"]["recall"],
    "F1-Score": report_seaswa["weighted avg"]["f1-score"],
}

# ====================
# Formatação final
# ====================

performance.set_index("Modelo", inplace=True)
performance_formatada = performance.sort_values(by="Acurácia", ascending=False)
performance_formatada = performance_formatada.applymap(lambda x: f"{(100 * x):.2f} %")

display(performance_formatada)
